In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

In [2]:
vest_ak_20 = gp.read_file("./raw-from-source/VEST/ak_2020/ak_2020.shp")

In [3]:
print(vest_ak_20.shape)

(441, 15)


In [4]:
vest_ak_20.head(2)

,DISTRICT,NAME,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGJAN,G20PRECBLA,G20PREIPIE,G20PREOFUE,G20USSRSUL,G20USSDGRO,G20USSOHOW,G20HALRYOU,G20HALDGAL,geometry
0,11-075,11-075 Palmer City No. 2,685,397,46,12,7,0,0,656,421,71,686,441,"POLYGON ((-149.13365 61.61026, -149.13361 61.6..."
1,12-220,12-220 Butte,1808,689,62,16,4,3,0,1761,673,96,1780,751,"POLYGON ((-148.52103 61.62682, -148.52020 61.6..."


Election results from Alaska Division of Elections (http://www.elections.alaska.gov/results/20GENR/index.php)
Precinct shapefile from Alaska Division of Elections (http://www.elections.alaska.gov/Core/districtmaps.php)

Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD. Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.

G20PRERTRU - Donald J. Trump (Republican Party)
G20PREDBID - Joseph R. Biden (Democratic Party)
G20PRELJOR - Jo Jorgensen (Libertarian Party)
G20PREGJAN - James G. "Jesse Ventura" Janos (Green Party)
G20PRECBLA - Don Blankenship (Conservative Party)
G20PREIPIE - Brock Pierce (Independent)
G20PREOFUE - Roque "Rocky" De La Fuente (Alliance Party)

G20USSRSUL - Dan Sullivan (Republican Party)
G20USSDGRO - Al Gross (Democratic Party)
G20USSOHOW - John Wayne Howe (Alaskan Independence Party)

G20HALRYOU - Don Young (Republican Party)
G20HALDGAL - Alyse S. Galvin (Democratic Party)

In [5]:
ak_2020 = pd.read_csv("./raw-from-source/Election_Results/resultsbyprecinct.txt",header=None)


In [6]:
ak_2020.head(1)

,0,1,2,3,4,5,6,7,8
0,HD99 Fed Overseas Absentee,Race Statistics,NP,NP,Number of Precincts,NP,Total,0,NaN


In [7]:
ak_2020.columns = ["Precinct","Race","Blank_1","Blank_2","Choice","Party","Type","Votes","Extra"]
ak_2020 = ak_2020 [["Precinct","Race","Choice","Party","Votes"]]

In [8]:
print(ak_2020["Race"].unique())

['Race Statistics' 'U.S. President / Vice President""' 'U.S. Senator""'
 'U.S. Representative""' 'House District 1""' 'Supreme Court - Carney""'
 'Court of Appeals - Wollenberg""' 'Superior Court JD4 - Peters""'
 'District Court JD4 - Christian""' 'District Court JD4 - Montgomery""'
 'Ballot Measure No. 1 - 19OGTX""' 'Ballot Measure No. 2 - 19AKBE""'
 'House District 2""' 'Senate District B""' 'House District 3""'
 'House District 4""' 'House District 5""' 'House District 6""'
 'Superior Court JD3 - Crosby""' 'Superior Court JD3 - Guidi""'
 'Superior Court JD3 - Henderson""' 'Superior Court JD3 - Lamoureux""'
 'Superior Court JD3 - Miller""' 'Superior Court JD3 - Reigh""'
 'Superior Court JD3 - Wells""' 'Superior Court JD3 - Woodman""'
 'District Court JD3 - Dickson""' 'District Court JD3 - Franciosi""'
 'District Court JD3 - Hanley""' 'District Court JD3 - Logue""'
 'District Court JD3 - McCrea""' 'District Court JD3 - Wallace""'
 'District Court JD3 - Washington""' 'Senate District D

In [9]:
race_list = ['U.S. President / Vice President""', 'U.S. Senator""','U.S. Representative""']
ak_2020 = ak_2020[ak_2020["Race"].isin(race_list)]

In [10]:
ak_2020["Choice"].unique()

array(['Number of Precincts for Race', 'Number of Precincts Reporting',
       'Registered Voters', 'Times Counted',
       'Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""',
       'Trump  Donald J. / Pence  Michael R.""', 'Gross  Al""',
       'Howe  John Wayne""', 'Sullivan  Dan""', 'Galvin  Alyse S.""',
       'Young  Don""'], dtype=object)

In [11]:
choice_list = ['Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""',
       'Trump  Donald J. / Pence  Michael R.""', 'Gross  Al""',
       'Howe  John Wayne""', 'Sullivan  Dan""', 'Galvin  Alyse S.""',
       'Young  Don""']
ak_2020 = ak_2020[ak_2020["Choice"].isin(choice_list)]

In [12]:
ak_2020_pvt = pd.pivot_table(ak_2020,index=["Precinct"],values=["Votes"],columns=["Choice"],aggfunc=sum)
ak_2020_pvt.columns=ak_2020_pvt.columns.droplevel(0)
ak_2020_pvt.reset_index(inplace=True,drop=False)

In [13]:
ak_2020_pvt.head(1)

Choice,Precinct,"Biden Joseph R. Jr. / Harris Kamala D.""""","Blankenship Don / Mohr William""""","De La Fuente Rocque ""Rocky"" / Richardson Darcy G.""""","Galvin Alyse S.""""","Gross Al""""","Howe John Wayne""""","Janos James G. ""Jesse Ventura"" / McKinney Cynthia""""","Jorgensen Jo / Cohen Jeremy ""Spike""""""","Pierce Brock / Ballard Karla""""","Sullivan Dan""""","Trump Donald J. / Pence Michael R.""""","Young Don"""""
0,01-446 Aurora,239,3,0,293,246,46,2,43,3,465,466,452


In [14]:
print(ak_2020_pvt.columns)

Index(['Precinct', 'Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Galvin  Alyse S.""', 'Gross  Al""', 'Howe  John Wayne""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""', 'Sullivan  Dan""',
       'Trump  Donald J. / Pence  Michael R.""', 'Young  Don""'],
      dtype='object', name='Choice')


In [15]:
column_changes_dict = {'Biden  Joseph R. Jr. / Harris  Kamala D.""':"G20PREDBID",
                       'Jorgensen  Jo / Cohen  Jeremy "Spike"""':"G20PRELJOR",
                       'Trump  Donald J. / Pence  Michael R.""':"G20PRERTRU",
                       'Blankenship  Don / Mohr  William""':"G20PRECBLA",
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""':"G20PREOFUE",
        'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""':"G20PREGJAN",
                       'Pierce  Brock / Ballard  Karla""':"G20PREIPIE", 
        'Gross  Al""':"G20USSDGRO", 
        'Howe  John Wayne""':"G20USSOHOW",
         'Sullivan  Dan""':"G20USSRSUL",
                       
       'Galvin  Alyse S.""':"G20HALDGAL",               
        'Young  Don""':"G20HALRYOU"
    
}

ak_2020_pvt = ak_2020_pvt.rename(columns=column_changes_dict)

In [16]:
print(ak_2020_pvt.head())

ak_2020_pvt = ak_2020_pvt[["Precinct",'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']]

Choice                 Precinct  G20PREDBID  G20PRECBLA  G20PREOFUE  \
0                01-446 Aurora          239           3           0   
1       01-455 Fairbanks No. 1           47           0           0   
2       01-465 Fairbanks No. 2           85           1           1   
3       01-470 Fairbanks No. 3          128           0           1   
4       01-475 Fairbanks No. 4          103           2           0   

Choice  G20HALDGAL  G20USSDGRO  G20USSOHOW  G20PREGJAN  G20PRELJOR  \
0              293         246          46           2          43   
1               50          47           8           1           2   
2               98          79          23           1          12   
3              141         128          13           2          11   
4              110          91          16           1           5   

Choice  G20PREIPIE  G20USSRSUL  G20PRERTRU  G20HALRYOU  
0                3         465         466         452  
1                0          83        

In [17]:
print(vest_ak_20.columns)

Index(['DISTRICT', 'NAME', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL', 'geometry'],
      dtype='object')


In [18]:
data_columns = ['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']

In [19]:
print("Printing differences below:")
for race in data_columns:
    if (sum(vest_ak_20[race])-sum(ak_2020_pvt[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ak_16[race])-sum(ak_2020_pvt[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ak_20[race]))+" votes")
        print("\tSOURCES: "+str(sum(ak_2020_pvt[race]))+" votes")
print("")
print("All other races are equal")

Printing differences below:

All other races are equal


Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD. Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.

In [20]:
races=['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']
for race in races:
    ak_2020_pvt[race] = ak_2020_pvt[race].astype(int)

In [21]:
to_allocate = list(ak_2020_pvt[ak_2020_pvt["Precinct"].str.contains("District")]["Precinct"])

In [22]:
print(to_allocate)

['District 1  - Question ', 'District 1 - Absentee ', 'District 1 - Early Voting ', 'District 10  - Question ', 'District 10 - Absentee ', 'District 10 - Early Voting ', 'District 11  - Question ', 'District 11 - Absentee ', 'District 11 - Early Voting ', 'District 12  - Question ', 'District 12 - Absentee ', 'District 12 - Early Voting ', 'District 13  - Question ', 'District 13 - Absentee ', 'District 13 - Early Voting ', 'District 14  - Question ', 'District 14 - Absentee ', 'District 14 - Early Voting ', 'District 15  - Question ', 'District 15 - Absentee ', 'District 15 - Early Voting ', 'District 16  - Question ', 'District 16 - Absentee ', 'District 16 - Early Voting ', 'District 17  - Question ', 'District 17 - Absentee ', 'District 17 - Early Voting ', 'District 18  - Question ', 'District 18 - Absentee ', 'District 18 - Early Voting ', 'District 19  - Question ', 'District 19 - Absentee ', 'District 19 - Early Voting ', 'District 2  - Question ', 'District 2 - Absentee ', 'Di

In [23]:
ak_2020_to_allocate = ak_2020_pvt[ak_2020_pvt["Precinct"].isin(to_allocate)]
ak_2020_precinct = ak_2020_pvt[~(ak_2020_pvt["Precinct"].isin(to_allocate)) & (ak_2020_pvt["Precinct"]!="HD99 Fed Overseas Absentee ")]
ak_2020_fed_overseas = ak_2020_pvt[ak_2020_pvt["Precinct"]=="HD99 Fed Overseas Absentee "]

In [24]:
print(sum(ak_2020_pvt["G20PREOFUE"]))

318


In [25]:
print(ak_2020_to_allocate.shape)
print(ak_2020_precinct.shape)
print(ak_2020_fed_overseas.shape)
print(ak_2020_pvt.shape)

(138, 13)
(441, 13)
(1, 13)
(580, 13)


In [26]:
ak_2020_precinct.columns

Index(['Precinct', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL'],
      dtype='object', name='Choice')

In [27]:
ak_2020_precinct.loc[:,"District"] = ak_2020_precinct.loc[:,"Precinct"].apply(lambda x: x[0:2])
ak_2020_to_allocate.loc[:,"col_names"]=ak_2020_to_allocate.loc[:,"Precinct"].str.split(" - ")
ak_2020_to_allocate.loc[:,"District"]=ak_2020_to_allocate.loc[:,"col_names"].apply(lambda x: x[0].strip()[9:].zfill(2))

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [28]:
print(len(ak_2020_to_allocate["District"].unique()))
print(len(ak_2020_precinct["District"].unique()))

40
40


In [29]:
to_dole_out_totals = pd.DataFrame(ak_2020_to_allocate.groupby(["District"]).sum())
to_dole_out_totals.reset_index(inplace=True,drop=False)

In [30]:
print(sum(to_dole_out_totals["G20PREOFUE"]))

136


In [31]:
print(to_dole_out_totals["District"].unique())

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40']


In [32]:
precinct_specific_totals = pd.DataFrame(ak_2020_precinct.groupby(["District"]).sum())
precinct_specific_totals.reset_index(inplace=True,drop=False)

In [69]:
races=['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']

for index, row in precinct_specific_totals.iterrows():
    for race in races:
        if (row[race]==0):
            race_district = row["District"]
            to_allocate = int(to_dole_out_totals.loc[to_dole_out_totals["District"]==race_district,race])
            if (to_allocate != 0):
                print("Weren't able to allocate "+str(to_allocate)+" votes in " +str(race)+" "+str(race_district))


Weren't able to allocate 4 votes in G20PREOFUE 21
Weren't able to allocate 3 votes in G20PREOFUE 30


In [34]:
print(precinct_specific_totals)

Choice District  G20PRERTRU  G20PREDBID  G20PRELJOR  G20PREGJAN  G20PRECBLA  \
0            01        1945        1170         126          23          13   
1            02        1943         674         130          17           9   
2            03        4074         683         174          23          19   
3            04        2790        1610         164          56          20   
4            05        2431        1256         129          40          12   
5            06        3531        1454         155          54          21   
6            07        2999         528         120          26          12   
7            08        4266         584         145          25          18   
8            09        4543         949         180          37          18   
9            10        4175         756         153          36          11   
10           11        3338         818         113          32           9   
11           12        4242         811         150 

In [35]:
races=['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']



#Create some new columns for each of these races to deal with the allocation
for race in races:
    add_var = race+"_add"
    rem_var = race+"_rem"
    floor_var = race+"_floor"
    ak_2020_precinct.loc[:,add_var]=0.0
    ak_2020_precinct.loc[:,rem_var]=0.0
    ak_2020_precinct.loc[:,floor_var]=0.0

#Iterate over the rows
#Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
for index, row in ak_2020_precinct.iterrows():
    for race in races:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #Grab the district
        county_id = row["District"]
        #Get the denominator for the allocation (the precinct vote totals)
        denom = precinct_specific_totals.loc[precinct_specific_totals["District"]==county_id][race]
        #Get one of the numerators, how many districtwide votes to allocate
        numer = to_dole_out_totals.loc[to_dole_out_totals["District"]==county_id][race]
        #Get the vote totals for this race in this precinct
        val = ak_2020_precinct.at[index,race]
        #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
        if (float(denom==0)):
            vote_share = 0
        else:
            vote_share = (float(val)/float(denom))*float(numer)
        ak_2020_precinct.at[index,add_var] = vote_share
        #Take the decimal remainder of the allocation
        ak_2020_precinct.at[index,rem_var] = vote_share%1
        #Take the floor of the allocation
        ak_2020_precinct.at[index,floor_var] = np.floor(vote_share)

#After the first pass through, get the sums of the races by district to assist in the rounding            
first_allocation = pd.DataFrame(ak_2020_precinct.groupby(["District"]).sum())

#Now we want to iterate district by district to work on rounding
county_list = list(to_dole_out_totals["District"].unique()) 

#Iterate over the district
for county in county_list:
    for race in races:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
        to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals["District"]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
        #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
        for index in ak_2020_precinct.loc[ak_2020_precinct["District"]==county][rem_var].nlargest(to_go).index:
            ak_2020_precinct.at[index,add_var] = np.ceil(ak_2020_precinct.at[index,add_var])

#Iterate over every race again
for race in races:
    add_var = race+"_add"
    #Round every allocation down to not add fractional votes
    ak_2020_precinct.loc[:,add_var]=np.floor(ak_2020_precinct.loc[:,add_var])
    ak_2020_precinct.loc[:,race]+=ak_2020_precinct.loc[:,add_var]

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying

In [36]:
print(precinct_specific_totals[precinct_specific_totals["G20PREOFUE"]==0][["District","G20PREOFUE"]])

Choice District  G20PREOFUE
20           21           0
29           30           0


In [38]:
print(ak_2020_precinct[ak_2020_precinct["District"].isin(["21","30"])])

Choice                 Precinct  G20PRERTRU  G20PREDBID  G20PRELJOR  \
175     21-600 Turnagain No. 1        337.0       708.0        18.0   
176     21-605 Sand Lake No. 1        692.0       684.0        27.0   
177     21-610 Sand Lake No. 2        518.0       795.0        35.0   
178     21-615 Sand Lake No. 3        485.0       526.0        20.0   
179        21-620 Lake Spenard        381.0       494.0        25.0   
180           21-625 Lake Hood        503.0       803.0        25.0   
181     21-630 Turnagain No. 2        281.0       506.0        18.0   
182     21-635 Turnagain No. 3        493.0       898.0        35.0   
248             30-200 Central       1151.0       350.0        40.0   
249             30-210 K-Beach       1606.0       480.0        60.0   
250         30-220 Kenai No. 1       1312.0       561.0        51.0   
251         30-230 Kenai No. 2        740.0       314.0        21.0   
252         30-240 Kenai No. 3        627.0       243.0        28.0   
253   

In [39]:
print(to_dole_out_totals[to_dole_out_totals["G20PREOFUE"]!=0][["District","G20PREOFUE"]])

Choice District  G20PREOFUE
1            02           4
2            03           5
4            05           4
5            06           1
6            07           4
7            08           3
8            09           3
9            10           3
10           11           4
12           13           3
13           14           4
14           15           6
15           16           9
16           17           8
17           18           3
18           19           4
19           20           3
20           21           4
21           22           2
22           23           6
23           24           4
24           25          13
25           26           1
26           27           3
27           28           2
28           29           1
29           30           3
30           31           4
31           32           1
32           33           6
33           34           2
35           36           2
36           37           1
37           38           6
38           39     

In [40]:
totals = pd.DataFrame(ak_2020_precinct.sum())
precinct_vote_total_across_districts = totals.T

In [41]:
print(precinct_vote_total_across_districts)
print(ak_2020_fed_overseas)

Choice                                           Precinct G20PRERTRU  \
0       01-446 Aurora 01-455 Fairbanks No. 1 01-465 Fa...   189892.0   

Choice G20PREDBID G20PRELJOR G20PREGJAN G20PRECBLA G20PREIPIE G20PREOFUE  \
0        153405.0     8896.0     2673.0     1127.0      825.0      311.0   

Choice G20USSRSUL G20USSDGRO  ... G20USSDGRO_floor G20USSOHOW_add  \
0        191060.0   145721.0  ...         101461.0         7214.0   

Choice G20USSOHOW_rem G20USSOHOW_floor G20HALRYOU_add G20HALRYOU_rem  \
0               208.0           7006.0        89007.0          219.0   

Choice G20HALRYOU_floor G20HALDGAL_add G20HALDGAL_rem G20HALDGAL_floor  
0               88788.0       108173.0          208.0         107965.0  

[1 rows x 50 columns]
Choice                     Precinct  G20PRERTRU  G20PREDBID  G20PRELJOR  \
579     HD99 Fed Overseas Absentee           59         373           1   

Choice  G20PREGJAN  G20PRECBLA  G20PREIPIE  G20PREOFUE  G20USSRSUL  \
579              0          

In [42]:
for index, row in precinct_vote_total_across_districts.iterrows():
    for race in races:
        if (row[race]==0):
            print("yes")

In [43]:
#Create some new columns for each of these races to deal with the allocation
races=['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']

for race in races:
    add_var = race+"_fed_add"
    rem_var = race+"_fed_rem"
    floor_var = race+"_fed_floor"
    ak_2020_precinct.loc[:,add_var]=0.0
    ak_2020_precinct.loc[:,rem_var]=0.0
    ak_2020_precinct.loc[:,floor_var]=0.0

for index, row in ak_2020_precinct.iterrows():
    for race in races:
            add_var = race+"_fed_add"
            rem_var = race+"_fed_rem"
            floor_var = race+"_fed_floor"
            #Get the denominator for the allocation (the precinct vote totals)
            denom = precinct_vote_total_across_districts.iloc[0][race]
            #Get one of the numerators, how many countywide votes to allocate
            numer = ak_2020_fed_overseas.iloc[0][race]
            #Get the vote totals for this race in this precinct
            val = ak_2020_precinct.at[index,race]
            #Get the vote share, the precincts % of total precinct votes in the county times votes to allocate
            vote_share = (float(val)/float(denom))*float(numer)
            ak_2020_precinct.at[index,add_var] = vote_share
            #Take the decimal remainder of the allocation
            ak_2020_precinct.at[index,rem_var] = vote_share%1
            #Take the floor of the allocation
            ak_2020_precinct.at[index,floor_var] = np.floor(vote_share)
            
#After the first pass through, get the sums of the races by county 
totals = pd.DataFrame(ak_2020_precinct.sum())
first_allocation = totals.T
 
for race in races:
    add_var = race+"_fed_add"
    rem_var = race+"_fed_rem"
    floor_var = race+"_fed_floor"
    #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
    to_go = int(np.round((int(ak_2020_fed_overseas.iloc[0][race])-first_allocation.iloc[0][floor_var])))
    #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
    for index in ak_2020_precinct[rem_var].nlargest(to_go).index:
        ak_2020_precinct.at[index,add_var] = np.ceil(ak_2020_precinct.at[index,add_var])

#Iterate over every race again
for race in races:
    add_var = race+"_fed_add"
    #Round every allocation down
    ak_2020_precinct.loc[:,add_var]=np.floor(ak_2020_precinct.loc[:,add_var])

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying

In [44]:
print("Printing issues below: ")
for race in races:
    add_var = race+"_fed_add"
    add_var_2 = race+"_add"
    diff_1 = int(ak_2020_fed_overseas.iloc[0][race]-ak_2020_precinct.loc[:,add_var].sum())
    diff_2 = int(to_dole_out_totals.loc[:,race].sum()-ak_2020_precinct.loc[:,add_var_2].sum())
    if((diff_1 != 0) & (diff_2 !=0)):
        print(diff_1)
        print(diff_2)
        print("Issue with "+ race)
print("")
print("All others match")

Printing issues below: 

All others match


In [45]:
print("Printing issues below: ")
for race in races:
    add_var = race+"_fed_add"
    add_var_2 = race+"_add"
    diff_1 = int(ak_2020_fed_overseas.iloc[0][race]-ak_2020_precinct.loc[:,add_var].sum())
    diff_2 = int(to_dole_out_totals.loc[:,race].sum()-ak_2020_precinct.loc[:,add_var_2].sum())
    if((diff_1 != 0) & (diff_2 !=0)):
        print(diff_1)
        print(diff_2)
        print("Issue with "+ race)
print("")
print("All others match")

Printing issues below: 

All others match


In [46]:
for race in races:
    add_var = race+"_fed_add"
    #add_var_2 = race+"_add"
    ak_2020_precinct.loc[:,race]+=ak_2020_precinct.loc[:,add_var]
    #ak_2020_precinct.loc[:,race]+=ak_2020_precinct.loc[:,add_var_2]

rem_races = ["G20PREGJAN","G20PRECBLA","G20PREIPIE","G20PREOFUE"]
    
#for race in rem_races:
    #add_var_2 = race+"_add"
    #ak_2020_precinct.loc[:,race]+=ak_2020_precinct.loc[:,add_var_2]  

In [47]:
print(sum(vest_ak_20["G20PREOFUE"]))

318


In [48]:
ak_2020_precinct.columns

Index(['Precinct', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL', 'District', 'G20PRERTRU_add',
       'G20PRERTRU_rem', 'G20PRERTRU_floor', 'G20PREDBID_add',
       'G20PREDBID_rem', 'G20PREDBID_floor', 'G20PRELJOR_add',
       'G20PRELJOR_rem', 'G20PRELJOR_floor', 'G20PREGJAN_add',
       'G20PREGJAN_rem', 'G20PREGJAN_floor', 'G20PRECBLA_add',
       'G20PRECBLA_rem', 'G20PRECBLA_floor', 'G20PREIPIE_add',
       'G20PREIPIE_rem', 'G20PREIPIE_floor', 'G20PREOFUE_add',
       'G20PREOFUE_rem', 'G20PREOFUE_floor', 'G20USSRSUL_add',
       'G20USSRSUL_rem', 'G20USSRSUL_floor', 'G20USSDGRO_add',
       'G20USSDGRO_rem', 'G20USSDGRO_floor', 'G20USSOHOW_add',
       'G20USSOHOW_rem', 'G20USSOHOW_floor', 'G20HALRYOU_add',
       'G20HALRYOU_rem', 'G20HALRYOU_floor', 'G20HALDGAL_add',
       'G20HALDGAL_rem', 'G20HALDGAL_floor', 'G20PRERTRU_fed_add',
       'G

In [49]:
ak_2020_precinct = ak_2020_precinct[['Precinct','District', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGJAN',
       'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL', 'G20USSDGRO',
       'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']]

In [50]:
print("Printing differences below:")
for race in data_columns:
    if (sum(vest_ak_20[race])-sum(ak_2020_precinct[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ak_20[race])-sum(ak_2020_precinct[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ak_20[race]))+" votes")
        print("\tSOURCES: "+str(sum(ak_2020_precinct[race]))+" votes")
print("")
print("All other races are equal")

Printing differences below:
G20PREOFUE has a difference of 7.0 votes
	VEST: 318 votes
	SOURCES: 311.0 votes

All other races are equal


In [51]:
print(vest_ak_20)

    DISTRICT                       NAME  G20PRERTRU  G20PREDBID  G20PRELJOR  \
0     11-075   11-075 Palmer City No. 2         685         397          46   
1     12-220               12-220 Butte        1808         689          62   
2     10-055         10-055 Lakes No. 2         614         172          39   
3     15-300          15-300 JBER No. 2         567         287          68   
4     28-110  28-110 Rabbit Creek No. 2        1147         755          37   
..       ...                        ...         ...         ...         ...   
436   40-034           40-034 Point Lay          37          25           8   
437   40-036             40-036 Selawik          61          64           1   
438   40-038            40-038 Shungnak          12          75           0   
439   40-040          40-040 Wainwright         167          66           5   
440   40-330              40-330 Hughes           5          52           4   

     G20PREGJAN  G20PRECBLA  G20PREIPIE  G20PREOFUE

In [52]:
print(ak_2020_precinct["District"].unique())
vest_ak_20["District_mod"] = vest_ak_20["DISTRICT"].str[0:2]
print(vest_ak_20["District_mod"].unique())

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40']
['11' '12' '10' '15' '28' '16' '17' '18' '19' '20' '21' '22' '23' '24'
 '25' '26' '27' '13' '14' '07' '08' '29' '30' '31' '32' '33' '34' '35'
 '36' '01' '02' '03' '04' '05' '06' '09' '37' '38' '39' '40']


In [53]:
diff_counties=[]
for i in data_columns:
    diff = ak_2020_precinct.groupby(["District"]).sum()[i]-vest_ak_20.groupby(["District_mod"]).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(i)
        print(diff[diff != 0].to_string(header=False))
print("All other races in all counties are equal")

G20PREDBID
32    1.0
36   -1.0
G20PREOFUE
21   -4.0
30   -3.0
G20USSDGRO
23   -1.0
36    1.0
All other races in all counties are equal


In [54]:
ak_shp = gp.read_file("./raw-from-source/Shapefiles/SW Proc Shape Files/2013-SW-Proc-Shape-files.shp")
print(ak_shp)

      ID          AREA DISTRICT  MEMBERS LOCKED                       NAME  \
0    173      1.565101   11-075      1.0   None   11-075 Palmer City No. 2   
1    181    286.270233   12-220      1.0   None               12-220 Butte   
2    182      3.675227   10-055      1.0   None         10-055 Lakes No. 2   
3     32     29.413515   15-300      1.0   None          15-300 JBER No. 2   
4    144      6.259140   28-110      1.0   None  28-110 Rabbit Creek No. 2   
..   ...           ...      ...      ...    ...                        ...   
436  461  10066.747000   40-034      1.0   None         40-034 POINT LAY\n   
437  462   4899.020990   40-036      1.0   None           40-036 SELAWIK\n   
438  463   1661.107910   40-038      1.0   None          40-038 SHUNGNAK\n   
439  464   9258.007810   40-040      1.0   None        40-040 WAINWRIGHT\n   
440  465      2.990466   40-330      1.0   None            40-330 HUGHES\n   

     POPULATION  IDEAL_VALU  DEVIATION  F_DEVIATIO  \
0        

In [58]:
ak_2020_precinct["join_col"]=ak_2020_precinct["Precinct"].str[0:6]
ak_shp["join_col"]=ak_shp["NAME"].str[0:6]

print(ak_shp["join_col"].value_counts())
print(ak_2020_precinct["join_col"].value_counts())

12-230    1
32-850    1
15-300    1
37-700    1
17-410    1
         ..
09-628    1
39-928    1
13-245    1
40-008    1
22-665    1
Name: join_col, Length: 441, dtype: int64
12-230    1
25-845    1
38-820    1
12-225    1
10-035    1
         ..
38-814    1
09-628    1
13-260    1
39-928    1
22-665    1
Name: join_col, Length: 441, dtype: int64


In [59]:
first_join = pd.merge(ak_2020_precinct,ak_shp,how="outer",on="join_col",indicator=True)
print(first_join["_merge"].value_counts())

both          441
left_only       0
right_only      0
Name: _merge, dtype: int64


In [60]:
vest_ak_20["join_col"]=vest_ak_20["DISTRICT"]

In [61]:
final_merge = pd.merge(vest_ak_20,first_join,how="outer",on="join_col",indicator="final_merge")
print(final_merge["final_merge"].value_counts())

both          441
left_only       0
right_only      0
Name: final_merge, dtype: int64


In [67]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff >0):
                if(diff>2):
                    print(i, "{:.>72}".format(df.iloc[j]["join_col"]), "(V)","{:.>5}".format(int(df.iloc[j][left_Data]))," (S){:.>5}".format(int(df.iloc[j][right_Data])),"(D):{:>5}".format(int(df.iloc[j][right_Data])-int(df.iloc[j][left_Data])))           
                #print(df.iloc[j]['countypct'])
                
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]["join_col"])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    #print("New max diff is: ", str(max_diff))
                    #print(df.iloc[j]['cty_pct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["join_col"])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    diff_list.sort()
    print(diff_list)

In [68]:
validater_row(final_merge,races)

G20USSDGRO ..................................................................17-415 (V) ..624  (S)..627 (D):    3
G20PREDBID ..................................................................17-420 (V) .1158  (S).1161 (D):    3
G20USSDGRO ..................................................................18-445 (V) ..624  (S)..621 (D):   -3
G20PREDBID ..................................................................20-540 (V) ..215  (S)..212 (D):   -3
G20HALDGAL ..................................................................20-570 (V) ..690  (S)..686 (D):   -4
G20USSDGRO ..................................................................23-740 (V) ..620  (S)..617 (D):   -3
G20HALDGAL ..................................................................28-145 (V) ..430  (S)..427 (D):   -3
G20HALDGAL ..................................................................34-410 (V) ..725  (S)..728 (D):    3
G20HALDGAL ..................................................................34-460 (V) 

In [64]:
both = final_merge[final_merge["final_merge"]=="both"]
vest_geoms = gp.GeoDataFrame(both,geometry="geometry_x",crs=vest_ak_20.crs)
source_geoms = gp.GeoDataFrame(both,geometry="geometry_y",crs=vest_ak_20.crs)
source_geoms = source_geoms.to_crs(3857)
vest_geoms = vest_geoms.to_crs(3857)
source_geoms["geometry_x"]=source_geoms.buffer(0)
vest_geoms["geometry_y"]=vest_geoms.buffer(0)
vals = source_geoms.geom_almost_equals(vest_geoms,decimal=0)
print(vals.value_counts())

True    441
dtype: int64


In [65]:
count = 0
area_list = []
for i in range(0,len(source_geoms)):
    diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])
    intersection = source_geoms.iloc[[i]].intersection(vest_geoms.iloc[[i]])
    area = float(diff.area/10e6)
    area_list.append(area)
    #print("Area is " + str(area))

    if (area > .1):
        count += 1
        name = source_geoms.iat[i,2]
        
        print(str(count)+") For " + name + " difference in area is " + str(area))
        if (intersection.iloc[0].is_empty):
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            base.set_title(name)
        else:
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            intersection.plot(color="green",ax=base)
            base.set_title(name)

In [66]:
df = pd.DataFrame(area_list)
print(df.shape)

print(str(len(df[df[0]==0]))+" precincts w/ a difference of 0 km^2")
print(str(len(df[(df[0]<.1) & (df[0]>0)]))+ " precincts w/ a difference between 0 and .1 km^2")
print(str(len(df[(df[0]<.5) & (df[0]>=.1)]))+ " precincts w/ a difference between .1 and .5 km^2")
print(str(len(df[(df[0]<1) & (df[0]>=.5)]))+ " precincts w/ a difference between .5 and 1 km^2")
print(str(len(df[(df[0]<2) & (df[0]>=1)]))+ " precincts w/ a difference between 1 and 2 km^2")
print(str(len(df[(df[0]<5) & (df[0]>=2)]))+ " precincts w/ a difference between 2 and 5 km^2")
print(str(len(df[(df[0]>=5)]))+ " precincts w/ a difference greater than 5 km^2")

(441, 1)
441 precincts w/ a difference of 0 km^2
0 precincts w/ a difference between 0 and .1 km^2
0 precincts w/ a difference between .1 and .5 km^2
0 precincts w/ a difference between .5 and 1 km^2
0 precincts w/ a difference between 1 and 2 km^2
0 precincts w/ a difference between 2 and 5 km^2
0 precincts w/ a difference greater than 5 km^2
